In [3]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, MetaData

In [4]:
%load_ext sql
%sql sqlite:///database/user_presets.db

In [25]:
%%sql
SELECT * 
FROM sqlite_schema

 * sqlite:///database/user_presets.db
Done.


type,name,tbl_name,rootpage,sql
table,users,users,2,"CREATE TABLE users ( id INTEGER NOT NULL, username VARCHAR, PRIMARY KEY (id))"
index,ix_users_username,users,3,CREATE UNIQUE INDEX ix_users_username ON users (username)
index,ix_users_id,users,4,CREATE INDEX ix_users_id ON users (id)
table,user_presets,user_presets,6,"CREATE TABLE user_presets ( id INTEGER NOT NULL, username VARCHAR, profilename VARCHAR, time_line VARCHAR, media_buyer VARCHAR, activer_days INTEGER, campaign VARCHAR, user_id INTEGER, PRIMARY KEY (id), FOREIGN KEY(user_id) REFERENCES users (id))"
index,ix_user_presets_username,user_presets,7,CREATE UNIQUE INDEX ix_user_presets_username ON user_presets (username)
index,ix_user_presets_profilename,user_presets,8,CREATE UNIQUE INDEX ix_user_presets_profilename ON user_presets (profilename)
index,ix_user_presets_id,user_presets,5,CREATE INDEX ix_user_presets_id ON user_presets (id)


In [26]:
%%sql
SELECT name
FROM sqlite_schema
WHERE type = "table"

 * sqlite:///database/user_presets.db
Done.


name
users
user_presets


In [27]:
%%sql
select *
from users
;

 * sqlite:///database/user_presets.db
Done.


id,username


In [42]:
%%sql
select *
from user_presets
;

 * sqlite:///database/user_presets.db
Done.


id,username,profilename,time_line,media_buyer,activer_days,campaign,user_id
1,None,LUKE_test_1,14 Days,Luke,15,LH_AppDev_DE_1101 #2524,None
2,None,LUKE_test_2,7 Days,Luke,15,LH_Aug17_RehabFacilities_VN_07cd,None
3,Luke,LUKE_test_3,Lifetime,Luke,14,B125_CosmeticSurgeryinMX_IN_0904,None


In [43]:
engine = create_engine('sqlite:///database/user_presets.db')

# Your SQL query
query = "SELECT * FROM user_presets"

# Execute the query and fetch the results into a Pandas DataFrame
df = pd.read_sql_query(query, engine)
df.head()

,id,username,profilename,time_line,media_buyer,activer_days,campaign,user_id
0,1,None,LUKE_test_1,14 Days,Luke,15,LH_AppDev_DE_1101 #2524,None
1,2,None,LUKE_test_2,7 Days,Luke,15,LH_Aug17_RehabFacilities_VN_07cd,None
2,3,Luke,LUKE_test_3,Lifetime,Luke,14,B125_CosmeticSurgeryinMX_IN_0904,None


In [36]:
from sqlalchemy import create_engine, Column, Integer, String, and_, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.orm.exc import NoResultFound

In [9]:
Base = declarative_base()
# class UserProfile(Base):
#     """
#     Represents a user profile with personalized settings and presets.

#     Attributes:
#     - id (int): Unique identifier for the user profile.
#     - username (str): Unique username associated with the user.
#     - profilename (str): Unique name given to the user profile.
#     - time_line (str): Timeline setting for the user profile.
#     - media_buyer (str): Media buyer setting for the user profile.
#     - activer_days (int): Number of active days setting for the user profile.
#     - campaign (str): Campaign setting for the user profile.

#     Table Name: 'user_presets'
#     """
#     __tablename__ = 'user_presets'

#     id = Column(Integer, primary_key=True, index=True)
#     username = Column(String, index=True)
#     profilename = Column(String, unique=True, index=True)
#     time_line = Column(String)
#     media_buyer = Column(String)
#     activer_days = Column(Integer)
#     campaign = Column(String)

/var/folders/_b/_rc6tjkn42n6t50xp7rhttps9db1hv/T/ipykernel_6610/2049283257.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [10]:
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True)
    profiles = relationship('UserProfile', back_populates='user')

class UserProfile(Base):
    __tablename__ = 'user_presets'
    # __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True, index=True)
    profilename = Column(String, index=True, unique=True)  # Make the combination of username and profilename unique
    time_line = Column(String)
    media_buyer = Column(String)
    activer_days = Column(Integer)
    campaign = Column(String)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('User', back_populates='profiles')

In [11]:
engine = create_engine('sqlite:///database/user_presets.db')
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)

In [12]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

def save_settings(session, user_name, profile_name, settings):
    """
    Save user profile settings to the database.

    Parameters:
    - session (Session): SQLAlchemy database session.
    - user_name (str): Unique username associated with the user.
    - profile_name (str): Unique name given to the user profile.
    - settings (dict): Dictionary containing user profile settings.

    Returns:
    None
    """
    try:
        # Query the database for the user with the given username
        user = session.query(User).filter_by(username=user_name).first()

        if user:
            # Check if a profile with the same name already exists for the user
            existing_profile = session.query(UserProfile).filter_by(user=user, profilename=profile_name).first()

            if existing_profile:
                # Update the existing profile with the new settings
                for key, value in settings.items():
                    setattr(existing_profile, key, value)
            else:
                # Create a new UserProfile object with the provided parameters and settings
                new_profile = UserProfile(profilename=profile_name, **settings)
                # Append the new profile to the user's profiles
                user.profiles.append(new_profile)

            # Commit the changes to the database
            session.commit()

        else:
            print(f"User with username '{user_name}' not found.")

    except IntegrityError as e:
        # Handle IntegrityError, e.g., log the error or notify the user
        print(f"Error: {e}")
        session.rollback()

    except Exception as e:
        # Handle other exceptions as needed
        print(f"Error: {e}")
        session.rollback()

    finally:
        # Close the session (if not using a context manager)
        session.close()

In [13]:
profile_name = "Luke_test_2"
           
user_name = "Luke"
settings = {
    "time_line": "14 Days",
    "media_buyer": "Luke",
    "activer_days":13,
    "campaign": "test_campaign"
}

save_settings(Session(), user_name, profile_name, settings)


User with username 'Luke' not found.


In [26]:
def save_settings(user_name, profile_name, settings):
    """
    Save user profile settings to the database.

    Parameters:
    - user_name (str): Unique username associated with the user.
    - profile_name (str): Unique name given to the user profile.
    - settings (dict): Dictionary containing user profile settings.

    Returns:
    None
    """
    # Create a new database session
    session = Session()

    # Create a new UserProfile object with the provided parameters and settings
    user_profile_obj = UserProfile(username=user_name, profilename=profile_name, **settings)
    
    # Add the UserProfile object to the session
    session.add(user_profile_obj)

    # Commit the changes to the database
    session.commit()

    # Close the session
    session.close()

In [44]:
profile_name = "Luke_test_2"
           
user_name = "Luke"
settings = {
    "time_line": "14 Days",
    "media_buyer": "Luke",
    "activer_days":13,
    "campaign": "test_campaign"
}

save_settings(user_name, profile_name, settings)


TypeError: save_settings() missing 1 required positional argument: 'settings'

In [1]:
import pandas as pd
df = pd.read_csv("data_for_dash_full.csv")

In [2]:
df.head()

,ACTIVITY_DATE,MEDIA_BUYER,CAMPAIGN,SPEND,REVENUE,DAILY_PROFIT,TOTAL_PROFIT,DAILY_RETURN,TOTAL_RETURN,LANDER_ARRIVALS,SERP_ARRIVALS,AD_CLICKS,ACCEPTED_CLICKS,ACCEPTANCE_RATE,SPEND_PER_ARRIVAL,REVENUE_PER_ARRIVAL,PROFIT_PER_ARRIVAL
0,2023-09-01,Luke,LH_HairTransplant_VT_0411,5.39,11.28,5.89,5.89,1.092764,1.092764,132,102,0,91.0,inf,0.040833,0.085455,0.044621
1,2023-09-02,Luke,LH_HairTransplant_VT_0411,8.97,14.09,5.12,11.01,0.570792,1.663556,192,129,0,100.0,inf,0.046719,0.073385,0.026667
2,2023-09-03,Luke,LH_HairTransplant_VT_0411,8.19,12.51,4.32,15.33,0.527473,2.191028,201,133,0,99.0,inf,0.040746,0.062239,0.021493
3,2023-09-04,Luke,LH_HairTransplant_VT_0411,6.07,11.47,5.40,20.73,0.889621,3.080650,164,108,0,80.0,inf,0.037012,0.069939,0.032927
4,2023-09-05,Luke,LH_HairTransplant_VT_0411,4.58,10.81,6.23,26.96,1.360262,4.440912,131,93,0,72.0,inf,0.034962,0.082519,0.047557
